# Bitcoin Linear Regression: DataFrame Construction
2 February 2018

This notebook builds the dataframe and exports as a pickle file for use in the EDA and Linear Regression notebooks.  

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import dateutil.parser
from pprint import pprint
import json
import time
import sys
import re
import datetime

# APIs
import quandl

In [2]:
# Quandl API Calls
df_price = pd.read_csv('https://www.quandl.com/api/v3/datasets/BNC3/GWA_BTC.csv?api_key=pvPBMBW8afR_HqVfio9o') # Price, volume
df_eth = pd.read_csv('https://www.quandl.com/api/v3/datasets/GDAX/ETH_USD.csv?api_key=pvPBMBW8afR_HqVfio9o') # ETH Price, volume
df_fees = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/TRFUS.csv?api_key=pvPBMBW8afR_HqVfio9o') # Txn fees
df_cost = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/CPTRA.csv?api_key=pvPBMBW8afR_HqVfio9o') # cost per txn
df_no = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/NTRAN.csv?api_key=pvPBMBW8afR_HqVfio9o') # num txns
df_noblk = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/NTRBL.csv?api_key=pvPBMBW8afR_HqVfio9o') # txns per block
df_blksz = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/AVBLS.csv?api_key=pvPBMBW8afR_HqVfio9o') # blk size
df_unq = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/NADDU.csv?api_key=pvPBMBW8afR_HqVfio9o') # unique addys
df_hash = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/HRATE.csv?api_key=pvPBMBW8afR_HqVfio9o') # hash rate
df_diff = pd.read_csv('https://www.quandl.com/api/v3/datasets/BCHAIN/DIFF.csv?api_key=pvPBMBW8afR_HqVfio9o') # difficulty

df_nasdaq = pd.read_csv('https://www.quandl.com/api/v3/datasets/NASDAQOMX/COMP.csv?api_key=pvPBMBW8afR_HqVfio9o') # NASDAQ Composite
df_nasdaq = df_nasdaq.rename(columns={'Trade Date': 'Date','Index Value':'Nasdaq'})
df_nasdaq = df_nasdaq.drop(['High','Low','Total Market Value','Dividend Market Value'], 1)

df_gold = pd.read_csv('https://www.quandl.com/api/v3/datasets/NASDAQOMX/QGLD.csv?api_key=pvPBMBW8afR_HqVfio9o') # Nasdaq GOLD Index
df_gold = df_gold.rename(columns={'Trade Date': 'Date','Index Value':'Gold'})
df_gold = df_gold.drop(['High','Low','Total Market Value','Dividend Market Value'], 1)

In [3]:
# Helper functions
def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

def list_to_average(list):
    try:
        avg = list[0]/list[1]
    except:
        avg = 0
    return avg

def to_log(num):
    return np.log(num)

In [4]:
df = df_price.drop('Open', 1)
df = df.drop(['High','Low'], 1)
df = df.rename(columns={'Close': 'BTCPrice','Volume':'BTCVol'})
df = df_eth.merge(df,how='inner',on='Date')
df = df.rename(columns={'Open': 'ETHPrice'})
df = df.drop(['High','Low'], 1)
df = df_fees.merge(df, how='inner', on='Date')
df = df.rename(columns={'Value': 'TxFees'})
df = df_cost.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'CostperTxn'})
df = df_no.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'NoTxns'})
df = df_noblk.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'NoperBlock'})
df = df_blksz.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'AvgBlkSz'})
df = df_unq.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'UniqueAddresses'})
df = df_hash.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'HashRate'})
df = df_diff.merge(df,how='inner',on='Date')
df = df.rename(columns={'Value': 'Difficulty'})

df = df_nasdaq.merge(df,how='inner',on='Date')
df = df_gold.merge(df,how='inner',on='Date')

ct = [i for i in reversed(range(len(df)))]
df['DateNum'] = ct 

df['Date'] = df['Date'].apply(to_date)
df['Date'] = pd.to_datetime(df['Date'])
df['Date2'] = df['Date']
df = df.set_index('Date2')

df['Month'] = df['Date'].dt.month
df['Quarter'] = df['Date'].dt.quarter
df['Weekday'] = df['Date'].dt.weekday


df = df[['BTCPrice','ETHPrice','BTCVol','TxFees','CostperTxn','NoTxns','NoperBlock','AvgBlkSz','UniqueAddresses',
         'HashRate','Difficulty','Nasdaq','Gold','DateNum','Date','Month','Quarter','Weekday']]
df_hist = df

print(df_hist.shape)
df_hist.info()

(418, 18)
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 418 entries, 2018-02-01 to 2016-05-26
Data columns (total 18 columns):
BTCPrice           418 non-null float64
ETHPrice           418 non-null float64
BTCVol             418 non-null float64
TxFees             418 non-null float64
CostperTxn         418 non-null float64
NoTxns             418 non-null float64
NoperBlock         418 non-null float64
AvgBlkSz           418 non-null float64
UniqueAddresses    418 non-null float64
HashRate           418 non-null float64
Difficulty         418 non-null float64
Nasdaq             418 non-null float64
Gold               418 non-null float64
DateNum            418 non-null int64
Date               418 non-null datetime64[ns]
Month              418 non-null int64
Quarter            418 non-null int64
Weekday            418 non-null int64
dtypes: datetime64[ns](1), float64(13), int64(4)
memory usage: 62.0 KB


In [5]:
df_hist.head(2)

,BTCPrice,ETHPrice,BTCVol,TxFees,CostperTxn,NoTxns,NoperBlock,AvgBlkSz,UniqueAddresses,HashRate,Difficulty,Nasdaq,Gold,DateNum,Date,Month,Quarter,Weekday
Date2,,,,,,,,,,,,,,,,,,
2018-02-01,9152.859852,1093.0,602147.243960,1.642340e+06,80.529148,204276.0,1745.948718,1.042831,481100.0,1.513976e+07,2.603077e+12,7385.86,180.29,417,2018-02-01,2,1,3
2018-01-31,10226.021308,1076.9,376228.456417,1.632514e+06,96.749249,236609.0,1416.820359,0.987509,531440.0,2.160975e+07,2.603077e+12,7411.48,181.17,416,2018-01-31,1,1,2


In [6]:
df_hist.tail(2)

,BTCPrice,ETHPrice,BTCVol,TxFees,CostperTxn,NoTxns,NoperBlock,AvgBlkSz,UniqueAddresses,HashRate,Difficulty,Nasdaq,Gold,DateNum,Date,Month,Quarter,Weekday
Date2,,,,,,,,,,,,,,,,,,
2016-05-27,486.962703,12.47,762209.731197,25184.016948,7.157532,220093.0,1630.318519,0.823084,364137.0,1.357376e+06,1.993121e+11,4933.50,168.45,1,2016-05-27,5,2,4
2016-05-26,458.279956,12.61,477365.253604,26820.960997,6.389729,247844.0,1795.971014,0.871457,397464.0,1.377192e+06,1.993121e+11,4901.77,172.67,0,2016-05-26,5,2,3


In [7]:
# run coinmarketcap_hist.py weekly to generate .json file (note file number)
mkt_cap = pd.read_json('data/coinmarketcap_hist2.json').T
mkt_cap['Date'] = mkt_cap.index
mkt_cap['Date'] = pd.to_datetime(mkt_cap['Date'],format='%Y%m%d',errors='ignore')
mkt_cap = mkt_cap.set_index('Date')
mkt_cap = mkt_cap[['BTC','ETH','Crypto Market Cap']]
mkt_cap.tail()

,BTC,ETH,Crypto Market Cap
Date,,,
2017-12-31,220903949498,69767510695,572573375760
2018-01-07,287582315011,106276577298,822534709108
2018-01-14,236679147035,132809805708,724968825266
2018-01-21,207286525403,107468021289,597208421609
2018-01-28,197709522984,113010221225,578768082167


In [8]:
# Download weekly from google trends from time period 2013-04-28 to present day
# https://trends.google.com/trends/explore?date=2013-04-28%202018-01-31&q=bitcoin
df_goog = pd.read_csv('data/20180131_GoogleTrendsSearchInterest.csv') # Google Trends "bitcoin" interest over time
df_goog = df_goog.iloc[2:]
df_goog = df_goog.rename(columns={'Category: All categories': 'Interest'})
df_goog['Date2'] = df_goog.index
df_goog['Date2'] = pd.to_datetime(df_goog['Date2'])
df_goog = df_goog.set_index('Date2')
# df_goog.info()

In [9]:
df_mc = pd.concat([mkt_cap, df_goog], axis=1)
df_mc.tail()

,BTC,ETH,Crypto Market Cap,Interest
2017-12-31,220903949498,69767510695,572573375760,48
2018-01-07,287582315011,106276577298,822534709108,45
2018-01-14,236679147035,132809805708,724968825266,56
2018-01-21,207286525403,107468021289,597208421609,38
2018-01-28,197709522984,113010221225,578768082167,30


In [10]:
df_all = pd.concat([df_hist, df_mc], axis=1)
df_all = df_all.fillna(method='ffill')
df_all = df_all.iloc[200:,:]
df_all.head()

,BTCPrice,ETHPrice,BTCVol,TxFees,CostperTxn,NoTxns,NoperBlock,AvgBlkSz,UniqueAddresses,HashRate,...,Gold,DateNum,Date,Month,Quarter,Weekday,BTC,ETH,Crypto Market Cap,Interest
2016-07-13,656.669626,10.55,871225.091442,34693.70,5.69,219034.0,1531.70,0.77,379937.0,1516960.89,...,227.19,32.0,2016-07-13,7.0,3.0,2.0,1.02017e+10,8.88582e+08,1.22546e+10,3
2016-07-14,656.132060,10.49,872834.409470,35233.50,5.06,218093.0,1677.63,0.81,367252.0,1379055.35,...,226.88,33.0,2016-07-14,7.0,3.0,3.0,1.02017e+10,8.88582e+08,1.22546e+10,3
2016-07-15,663.841255,11.50,756558.628070,38613.99,5.49,220655.0,1553.90,0.79,367953.0,1506352.77,...,224.00,34.0,2016-07-15,7.0,3.0,4.0,1.02017e+10,8.88582e+08,1.22546e+10,3
2016-07-17,663.841255,11.50,756558.628070,38613.99,5.49,220655.0,1553.90,0.79,367953.0,1506352.77,...,224.00,34.0,2016-07-15,7.0,3.0,4.0,1.0473e+10,9.35944e+08,1.28827e+10,2
2016-07-18,680.149570,11.35,752528.346010,32557.41,7.59,171013.0,1147.73,0.70,330330.0,1580609.59,...,224.43,35.0,2016-07-18,7.0,3.0,0.0,1.0473e+10,9.35944e+08,1.28827e+10,2


In [11]:
df_all = df_all[['BTCPrice','ETHPrice','BTCVol','Crypto Market Cap', 'CostperTxn','TxFees','NoTxns','AvgBlkSz','UniqueAddresses','HashRate','Difficulty','Nasdaq','Gold','Interest','DateNum','Quarter','Month','Weekday']]
df_all = pd.DataFrame(df_all,dtype=np.float64) # convert all values to float64


In [12]:
# add log columns
df_all['logBTCPrice'] = df_all['BTCPrice'].apply(to_log)
df_all['logNasdaq'] = df_all['Nasdaq'].apply(to_log)
df_all['logETHPrice'] = df_all['ETHPrice'].apply(to_log)
df_all['logGold'] = df_all['Gold'].apply(to_log)
df_all['logCrypto Market Cap'] = df_all['Crypto Market Cap'].apply(to_log)
df_all['logInterest'] = df_all['Interest'].apply(to_log)
df_all['logCostperTxn'] = df_all['CostperTxn'].apply(to_log)
df_all['logTxFees'] = df_all['TxFees'].apply(to_log)
df_all['logNoTxns'] = df_all['NoTxns'].apply(to_log)
df_all['logAvgBlkSz'] = df_all['AvgBlkSz'].apply(to_log)
df_all['logUniqueAddresses'] = df_all['UniqueAddresses'].apply(to_log)
df_all['logHashRate'] = df_all['HashRate'].apply(to_log)
df_all['logBTCVol'] = df_all['BTCVol'].apply(to_log)
df_all['logDifficulty'] = df_all['Difficulty'].apply(to_log)

df_all.columns

Index(['BTCPrice', 'ETHPrice', 'BTCVol', 'Crypto Market Cap', 'CostperTxn',
       'TxFees', 'NoTxns', 'AvgBlkSz', 'UniqueAddresses', 'HashRate',
       'Difficulty', 'Nasdaq', 'Gold', 'Interest', 'DateNum', 'Quarter',
       'Month', 'Weekday', 'logBTCPrice', 'logNasdaq', 'logETHPrice',
       'logGold', 'logCrypto Market Cap', 'logInterest', 'logCostperTxn',
       'logTxFees', 'logNoTxns', 'logAvgBlkSz', 'logUniqueAddresses',
       'logHashRate', 'logBTCVol', 'logDifficulty'],
      dtype='object')

In [13]:
# pickle the consolidate DataFrame
df_all.to_pickle('data/benson_btcsentiment_df.pkl')

## The following data was scraped but not used in the analysis at this stage, but may be incorporated in the future

In [14]:
# Coinmarketcap: Current market cap information by top coin
url = 'https://coinmarketcap.com/all/views/all/'
response=requests.get(url)
page=response.text
soup=BeautifulSoup(page,"lxml")

tables=soup.find_all("table")

rows=[row for row in tables[0].find_all('tr')]
df_curr = pd.read_html(tables[0].prettify())[0]
df.to_pickle('data/benson_btcsentiment_dfcurr.pkl')
df_curr.head()

,#,Name,Symbol,Market Cap,Price,Circulating Supply,Volume (24h),% 1h,% 24h,% 7d
0,1,BTC Bitcoin,BTC,"$145,848,576,420",$8660.00,16841637,"$12,449,800,000",0.99%,-5.86%,-21.86%
1,2,ETH Ethereum,ETH,"$86,813,035,306",$891.55,97372935,"$6,520,820,000",0.07%,-13.44%,-15.11%
2,3,XRP Ripple,XRP,"$33,369,029,357",$0.855414,"39,009,215,838 *","$3,375,850,000",1.90%,-12.85%,-29.48%
3,4,BCH Bitcoin Cash,BCH,"$19,641,624,795",$1159.08,16945875,"$878,444,000",0.24%,-9.88%,-27.43%
4,5,ADA Cardano,ADA,"$9,392,392,427",$0.362262,"25,927,070,538 *","$1,290,900,000",1.10%,-16.15%,-41.38%


In [15]:
# Coinmarketcap scraping: Bitcoin by time period.  When updating, be sure to download for period 2013-04-28 to present
url = 'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20130428&end=20180131'
response=requests.get(url)
page=response.text
soup=BeautifulSoup(page,"lxml")
tables=soup.find_all("table")

rows=[row for row in tables[0].find_all('tr')]
df = pd.read_html(tables[0].prettify())[0]
df['Date']=df['Date'].apply(to_date)
df = df.set_index('Date')
df.to_pickle('data/benson_btcsentiment_dfts.pkl')
df.head()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2018-01-31,10108.2,10381.6,9777.42,10221.1,8041160000,170183000000
2018-01-30,11306.8,11307.2,10036.20,10106.3,8637860000,190339000000
2018-01-29,11755.5,11875.6,11179.20,11296.4,7107360000,197871000000
2018-01-28,11475.3,12040.3,11475.30,11786.3,8350360000,193133000000
2018-01-27,11174.9,11614.9,10989.20,11440.7,7583270000,188054000000


In [16]:
df.tail()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2013-05-02,116.38,125.60,92.28,105.21,-,1292190000
2013-05-01,139.00,139.89,107.72,116.99,-,1542820000
2013-04-30,144.00,146.93,134.05,139.00,-,1597780000
2013-04-29,134.44,147.49,134.00,144.54,-,1491160000
2013-04-28,135.30,135.98,132.10,134.21,-,1500520000


In [17]:
# Coinmarketcap scraping: Ethereum. When updating, be sure to download for period 2013-04-28 to present
url = 'https://coinmarketcap.com/currencies/ethereum/historical-data/?start=20130428&end=20180131'
response=requests.get(url)
page=response.text
soup=BeautifulSoup(page,"lxml")
tables=soup.find_all("table")

rows=[row for row in tables[0].find_all('tr')]
df_eth = pd.read_html(tables[0].prettify())[0]
df_eth['Date']=df_eth['Date'].apply(to_date)
df_eth = df_eth.set_index('Date')
df_eth.to_pickle('data/benson_btcsentiment_dftseth.pkl')
df_eth.head()

,Open,High,Low,Close,Volume,Market Cap
Date,,,,,,
2018-01-31,1071.09,1128.66,1034.50,1118.31,3757560000,104231000000
2018-01-30,1184.13,1184.63,1058.97,1071.13,4107860000,115207000000
2018-01-29,1246.70,1256.70,1169.08,1182.36,3715080000,121269000000
2018-01-28,1111.78,1257.77,1111.78,1246.01,5372330000,108122000000
2018-01-27,1055.75,1121.98,1042.12,1107.07,3002870000,102651000000


In [18]:
# a script was written to collect twitter sentiment data and coinmarketcap market cap data in real time (every 5 mins)
# as historical twitter sentiment analysis data could not be located.  One week of data has been collected to date, 
# but is very noisy.  To continue collecting data by running the script as a daemon and incorporate in a future
# analysis.  See the script at btcpricesentiment*.py
btcsa = pd.read_json('data/btcpricesentiment_mc7.json',convert_axes=False).T # connect to most recent active file

btcsa['Date'] = btcsa.index
btcsa['Date'] = pd.to_datetime(btcsa['Date'])
btcsa['Bitcoin_S'] = btcsa['bitcoin_S'].apply(list_to_average)
btcsa['Ethereum_S'] = btcsa['ethereum_S'].apply(list_to_average)
btcsa['Blockchain_S'] = btcsa['blockchain_S'].apply(list_to_average)
btcsa = btcsa[['Date','Bitcoin_S','Ethereum_S','Blockchain_S','BTC','ETH','LTC','ADA','EOS','Crypto Market Cap']]
btcsa = btcsa.sort_values('Date')
btcsa = btcsa.set_index('Date')
btcsa.to_pickle('data/benson_btcsentiment_dfs.pkl')
print(btcsa.shape)
btcsa.head(2)

(286, 9)


,Bitcoin_S,Ethereum_S,Blockchain_S,BTC,ETH,LTC,ADA,EOS,Crypto Market Cap
Date,,,,,,,,,
2018-01-28 18:13:00,0.218750,-0.173106,0.437179,"$193,540,756,417","$116,129,045,529","$10,144,386,235","$16,631,593,500","$9,254,599,532","$576,359,759,424"
2018-01-28 18:14:00,0.472064,0.472606,0.395227,"$193,540,756,417","$116,129,045,529","$10,144,386,235","$16,631,593,500","$9,254,599,532","$576,359,759,424"


In [19]:
# Bitcoin futures scraping.  As futures began trading in December 2017, it appears there is not yet enough information
# to incorporate into the analysis at this time.  To revisit in the future.  
url = 'http://www.cmegroup.com/trading/equity-index/us-index/bitcoin.html'
response=requests.get(url)
page=response.text
soup=BeautifulSoup(page,"lxml")

tables=soup.find_all("table")

rows=[row for row in tables[0].find_all('tr')]
df = pd.read_html(tables[0].prettify())[0]
df = df[:5]
df

,Month,Charts,Last,Change,Prior Settle,Open,High,Low,Volume,Hi / Low Limit,Updated
0,FEB 2018,Show Price Chart,8590.0,-500.0,9090.0,8890.0,9075.0,7750.0,2535.0,10905 / 7275,15:54:43 CT 02 Feb 2018
1,MAR 2018,Show Price Chart,8490.0,-595.0,9085.0,8770.0,9055.0,7750.0,324.0,10915 / 7285,15:14:23 CT 02 Feb 2018
2,APR 2018,Show Price Chart,8695.0,-425.0,9120.0,8695.0,8695.0,8695.0,7.0,10950 / 7320,14:41:34 CT 02 Feb 2018
3,JUN 2018,Show Price Chart,8800.0,-490.0,9290.0,8080.0,9005.0,8080.0,20.0,11010 / 7380,15:20:57 CT 02 Feb 2018
4,Legend: Options Price Chart About This Report,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# TWITTER'S MAIN API FOR SENTIMENT ANALYSIS IS UTILIZED AS FOLLOWS:
# import pandas
# import json

# from tweepy import Stream
# from tweepy.streaming import StreamListener

# class MyListener(StreamListener):

#     def on_data(self, data):
#         try:
#             with open('bitcoin.json', 'a') as f:
#                 f.write(data)
#                 return True
#         except BaseException as e:
#             print("Error on_data: %s" % str(e))
#         return True

#     def on_error(self, status):
#         print(status)
#         return True

# twitter_stream = Stream(auth, MyListener())
# twitter_stream.filter(track=['#bitcoin'])